In [23]:
import pandas as pd
import numpy as np
from PIL import Image
import requests
from tqdm import tqdm
import os

df_anime = pd.read_csv('../data/03_primary/anime_final.csv', sep='|')

df_anime.head(3)

,anime_id,name,type,episodes,mal_score,members,studio,release-season,release-year,release-date,...,themes,demographics,synopsis,image_url,rating,va_list,staff_list,recommended_review_count,mixedfeelings_review_count,notrecommended_review_count
0,32281,Kimi no Na wa.,Movie,1,9.37,200630,['CoMix Wave Films'],summer,2016.0,NaN,...,[],NaN,"Mitsuha Miyamizu, a high school girl, yearns t...",https://cdn.myanimelist.net/images/anime/5/870...,PG-13 - Teens 13 or older,"['Kamishiraishi, Mone', 'Kamiki, Ryunosuke', '...","['Bezerra, Wendel', 'Kawamura, Genki', 'Itou, ...",808.0,88.0,50.0
1,5114,Fullmetal Alchemist: Brotherhood,TV,64,9.26,793665,['Bones'],spring,2009.0,NaN,...,['Military'],Shounen,After a horrific alchemy experiment goes wrong...,https://cdn.myanimelist.net/images/anime/1208/...,R - 17+ (violence & profanity),"['Park, Romi', 'Kugimiya, Rie', 'Miki, Shinich...","['Cook, Justin', 'Maruyama, Hiroo', 'Yonai, No...",912.0,59.0,39.0
2,28977,Gintama°,TV,51,9.25,114262,['Bandai Namco Pictures'],spring,2015.0,NaN,...,"['Gag Humor', 'Historical', 'Parody', 'Samurai']",Shounen,"Gintoki, Shinpachi, and Kagura return as the f...",https://cdn.myanimelist.net/images/anime/3/720...,PG-13 - Teens 13 or older,"['Sugita, Tomokazu', 'Kugimiya, Rie', 'Sakaguc...","['Miyawaki, Chizuru', 'Takamatsu, Shinji', 'Yo...",79.0,3.0,1.0


In [24]:
print(df_anime['image_url'].isna().sum())
df_anime['image_url'] = df_anime['image_url'].fillna("No URL")
print(df_anime['image_url'].isna().sum())

1131
0


In [70]:
if os.path.isfile('../data/05_model_input/missing_anime.csv'):
    df_anime = pd.read_csv('../data/05_model_input/missing_anime.csv', sep='|')
    for i in tqdm(range(len(df_anime))):
        url = df_anime['image_url'][i]
        if url == "No URL":
            pass
        else:
            img = Image.open(requests.get(url, stream=True).raw)
            # Export image
            filepath = f"../data/05_model_input/images/{df_anime['anime_id'][i]}.jpg"
            img.save(filepath)


100%|██████████| 11161/11161 [1:40:46<00:00,  1.85it/s] 


In [71]:
loaded_img_list = os.listdir('../data/05_model_input/images')
loaded_img_df = pd.DataFrame(loaded_img_list, columns=['loaded_img_path'])
loaded_img_df['anime_id']  = loaded_img_df.apply(lambda x: os.path.splitext(x['loaded_img_path'])[0], axis=1).astype(int)
display(loaded_img_df.head())

print(f'# of loaded anime images: {len(loaded_img_list)}')
print(f'Total # of anime images:s: {len(df_anime["image_url"])}')
print(f'# of remaining anime ids to scrape:{len(df_anime["image_url"])-len(loaded_img_list)}')


,loaded_img_path,anime_id
0,1.jpg,1
1,100.jpg,100
2,1000.jpg,1000
3,10000.jpg,10000
4,10003.jpg,10003


# of loaded anime images: 11095
Total # of anime images:s: 11161
# of remaining anime ids to scrape:66


In [72]:
# Export remaining anime_id list
# missing_anime = np.setdiff1d(df_anime['anime_id'], loaded_img_df['anime_id']).astype(int)
missing_anime = df_anime[~(df_anime['anime_id'].isin(loaded_img_df['anime_id']))][['anime_id','image_url']]
print(missing_anime)
print(f'Length of missing_anime list: {len(missing_anime)}')
missing_anime.to_csv('../data/05_model_input/missing_anime.csv',sep='|', index=False)

       anime_id image_url
0         32983    No URL
1         32995    No URL
2            21    No URL
3         31933    No URL
4         10937    No URL
...         ...       ...
11142     34492    No URL
11146     34491    No URL
11147     34312    No URL
11149     34388    No URL
11152     34399    No URL

[1131 rows x 2 columns]
Length of missing_anime list: 1131
